## 注意(attention)！开始做前必读项！
所有的代码一定要在这个文件里面编写，不要自己创建一个新的文件
对于提供的数据集，不要改存储地方，也不要修改文件名和内容
不要重新定义函数（如果我们已经定义好的），按照里面的思路来编写。当然，除了我们定义的部分，如有需要可以自行定义函数或者模块
写完之后，重新看一下哪一部分比较慢，然后试图去优化。一个好的习惯是每写一部分就思考这部分代码的时间复杂度和空间复杂度，AI工程是的日常习惯！
这次作业很重要，一定要完成！ 相信会有很多的收获！

In [ ]:
import numpy as np
import pandas as pd
from gensim import models
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.externals import joblib

加载训练集文件和测试集文件

In [21]:
data_text,data=[],[]
stopWords=[]

def load_data():
    '''
    函数说明：该函数用于加载数据集
    return: 
        -data: 表示所有数据拼接的原始数据
        -data_text: 表示数据集中的特征数据集
        -datatext: 表示经过分词之后的特征数据集
        -stopWords: 表示读取的停用词
    '''
    print('load Pre_process')
    data = pd.concat([
        pd.read_csv('train_clean.tsv', sep='\t'),
        pd.read_csv('dev_clean.tsv', sep='\t'),
        pd.read_csv('test_clean.tsv', sep='\t')
        ])
    print("读取数据集完成")
    data_text = list(data.text)  # .apply(lambda x: x.split(' '))
    datatext = []
    for i in range(len(data_text)):
        datatext.append(data_text[i].split(' '))
    stopWords = open('stopwords.txt').readlines()
    print("取停用词完成")
    return data, data_text,datatext, stopWords


In [22]:
data, data_text, datatext,stopWords=load_data()

load Pre_process
读取数据集完成
取停用词完成


In [13]:
def trainer_tfidf():
    '''
    函数说明：该函数用于训练tfidf的词向量
    return: 
        -tfidf: 表示经过TF-ID模型训练出的词向量
    '''
    print('train tfidf_embedding')
    # 训练tfidf的词向量
    # Convert a collection of raw documents to a matrix of TF-IDF features.
    count_vect = TfidfVectorizer(
        stop_words=stopWords, max_df=0.6, ngram_range=(1, 2))
    tfidf = count_vect.fit(data.text)
    return tfidf


train tfidf_embedding


/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['exp', 'lex', 'sub', 'sup', '①①', '①②', '①③', '①④', '①⑤', '①⑥', '①⑦', '①⑧', '①⑨', '①ａ', '①ｂ', '①ｃ', '①ｄ', '①ｅ', '①ｆ', '①ｇ', '①ｈ', '①ｉ', '①ｏ', '②①', '②②', '②③', '②④', '②⑤', '②⑥', '②⑦', '②⑧', '②⑩', '②ａ', '②ｂ', '②ｃ', '②ｄ', '②ｅ', '②ｆ', '②ｇ', '②ｈ', '②ｉ', '②ｊ', '③①', '③⑩', '③ａ', '③ｂ', '③ｃ', '③ｄ', '③ｅ', '③ｆ', '③ｇ', '③ｈ', '④ａ', '④ｂ', '④ｃ', '④ｄ', '④ｅ', '⑤ａ', '⑤ｂ', '⑤ｄ', '⑤ｅ', '⑤ｆ', '一一', '一个', '一些', '一切', '一则', '一方面', '一旦', '一来', '一样', '一番', '一直', '一般', '万一', '上下', '不仅', '不但', '不光', '不单', '不只', '不如', '不怕', '不惟', '不成', '不拘', '不比', '不然', '不特', '不独', '不管', '不论', '不过', '不问', '与其', '与否', '与此同时', '两者', '为了', '为什么', '为何', '为着', '乃至', '之一', '之所以', '之类', '乌乎', '也好', '也就是说', '也罢', '于是', '于是乎', '云云', '人家', '什么', '什么样', '从而', '他人', '他们', '以便', '以免', '以及', '以至', '以至于', '以致', '任何', '任凭', '似的', '但是', '何况',

KeyboardInterrupt: 

In [ ]:
tfidf = trainer_tfidf()

In [ ]:
根据数据集训练word2vec的词向量

In [19]:
def trainer_w2v():
    '''
    函数说明：该函数基于Word2vec模型训练词向量
    return: 
        -w2v: 表示经过word2vec模型训练出的词向量
    '''
    print('train word2vec Embedding')
        # 训练w2v的词向量
    w2v = models.Word2Vec(min_count=2,
                                window=3,
                                size=300,
                                sample=6e-5,
                                alpha=0.03,
                                min_alpha=0.0007,
                                negative=15,
                                workers=4,
                                iter=10,
                                max_vocab_size=50000)
        
    w2v.build_vocab(datatext)

    w2v.train(datatext,
                       total_examples=w2v.corpus_count,
                       epochs=15,
                       report_delay=1)
        
    print('train fast_embedding')
    return w2v

In [20]:
w2v = trainer_w2v()

train word2vec Embedding


NameError: name 'datatext' is not defined

In [ ]:
def trainer_fasttext():
    '''
    函数说明：该函数基于FastText模型训练词向量
    return: 
        -fast: 表示经过FastText模型训练出的词向量
    '''
    fast = models.FastText(datatext,
                                size=300,  # 向量维度
                                window=2,  # 移动窗口
                                alpha=0.03,
                                min_count=2,  # 对字典进行截断，小于该数的则会被切掉,增大该值可以减少词表个数
                                iter=10,  # 迭代次数
                                min_n=1,
                                max_n=3,
                                word_ngrams=2,
                                max_vocab_size=50000)
    return fast

In [ ]:
fast= trainer_fasttext()

保存生成的词向量

In [ ]:
def saver():
    '''
    函数说明：该函数存储训练好的模型
    '''
    print('save tfidf model')
    joblib.dump(tfidf, 'tfidf_model')
    print('save word2vec model')
    w2v.save('w2v_model')
    print('save fast model')
    fast.save('fast_model')
    
saver()
    

In [ ]:
加载模型 

In [ ]:
def load():
    '''
    函数说明：该函数加载训练好的模型
    '''
    print('load tfidf_embedding model')
    tfidf = joblib.load('tfidf_model')
    print('load w2v_embedding model')
    w2v = models.KeyedVectors.load('w2v_model')
    print('load fast_embedding model')
    fast = models.FastText.load('fast_model')
load()